# Discogs Functions

In [ ]:
## Basic stuff
%load_ext autoreload
%autoreload
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))
display(HTML("""<style>div.output_area{max-height:10000px;overflow:scroll;}</style>"""))

## Python Version
import sys
print("Python: {0}".format(sys.version))

from ioUtils import saveJoblib, loadJoblib, saveFile, getFile
import urllib
from urllib.parse import quote

from discogsBase import discogs
from discogsUtils import discogsUtils
from collection import collections
from artist import artist
from timeUtils import clock, elapsed
from fsUtils import moveFile, setFile, setDir, setSubDir, isFile, isDir, mkDir
from fileUtils import getFileBasics, getBasename
from artists import artists
from artist import artist
from albums import albums
from album import album, albumURLInfo
from time import sleep

import datetime as dt
start = dt.datetime.now()
print("Notebook Last Run Initiated: "+str(start))

In [ ]:
%load_ext autoreload
%autoreload
disc = discogs()
arts = artists(disc)
art  = artist()
albs = albums(disc)
alb  = album()
dutils = discogsUtils()

# Artist Section

# Parse Artists (Parallel)

In [ ]:
for modVal in range(50):
    parseArtists(modVal)

In [ ]:
%load_ext autoreload
%autoreload

from artist import artist
from artists import artists
arts = artists(disc)

def parseArtists(modVal, force=False, doExtra=False):
    #arts.parseArtistModValFiles(modVal, force=force)
    arts.parseArtistModValExtraFiles(modVal)


def parseArtistsParallel(nProc=3, force=False):
    pool = Pool(processes=nProcs)
    result = pool.map_async(parseArtists, range(100)) 
    start, cmt = clock("Processing {0} modVals using {1} processes".format(modVals, nProcs))
    while not result.ready():
        if force is True:
            time.sleep(10)
        else:
            time.sleep(1)
    print("")
    
    elapsed(start, cmt)
    return result.get()

start,cmt=clock("Parsing All Artists")
start2,cmt2=clock("Parallel Artist Parsing")
#parseArtistsParallel()
elapsed(start2, cmt2)
#arts.buildMetadata(force=False, doAlbums=False)
elapsed(start, cmt)

# Download Artist Albums

In [ ]:
disc.getDiagnosticAlbumIDs()

In [ ]:
%load_ext autoreload
%autoreload
from albums import albums
disc = discogs()
albs = albums(disc)
albs.downloadAlbumsFromArtists(maxAlbums=25, mediaTypes=None, rand=True)

# Download Searched Artist

In [ ]:
%load_ext autoreload
%autoreload
from albums import albums
disc = discogs()
arts = artists(disc)

In [ ]:
arts.searchDiscogForArtist("Melanie Sparks")

# Download Old Artist

In [ ]:
from time import ctime
from os import stat
from os.path import getctime
from timeUtils import getDateTime
from searchUtils import findDirs, findExt

artistIDs = {}
for dirval in findDirs("/Volumes/Music/Discog/artists/"):
    print(dirval,end="\t")
    for ifile in findExt(dirval, ext=".p"):
        mtime = ctime(getctime(ifile))
        dtime = getDateTime(mtime)        
        if dtime.year < 2019:
            artistIDs[ifile] = True
            continue
            artData = arts.getData(ifile)
            ID  = artData.ID.ID
            url = artData.url.url
            artistIDs[ID] = url
    print(len(artistIDs))

In [ ]:
artistIDtoName = disc.getArtistIDToNameData()
artistIDtoRef  = disc.getArtistIDToRefData()

In [ ]:
from fileUtils import getBaseFilename

In [ ]:
for ifile,artistURL in artistIDs.items():
    artistID = getBaseFilename(ifile)
    try:
        name = artistIDtoName[artistID]
    except:
        continue
    try:
        ref = artistIDtoRef[artistID]
    except:
        continue
        
    #print(artistID,name,ref,ifile)
    #1/0
        
    if len(name) != len(name.encode()):
        print(artistID,'\t',name)
        url = arts.getArtistURL(ref)
        savename = arts.getArtistSavename(artistID)
        try:
            arts.downloadArtistURL(url=url, savename=savename, force=True, sleeptime=1.75)
        except:
            continue

In [ ]:
arts.parseArtistModValFiles(38, force=True)
#arts.buildMetadata

In [ ]:
db38 = disc.getArtistsDBModValData(38)

In [ ]:
db38['216138'].artist.get()

In [ ]:
#[db38[x].artist.get()['name'] for x in db38.keys()]
discDB = disc.getArtistIDToNameData()

In [ ]:
discDB['216138']

In [ ]:
%load_ext autoreload
%autoreload

from artists import artists
disc = discogs()
arts = artists(disc)
arts.buildMetadata(doAlbums=False, force=True)

# Download Most Common Artist Albums

In [ ]:
from collections import Counter
db = Counter(disc.getCollectionRefCountsData())
coreAlbumsRefs = disc.getArtistIDCoreAlbumRefs()
artistMetaData = disc.getAlbumArtistMetaData()
albumDB = disc.getAlbumIDToRefData()

In [ ]:
toget = {}
for item in db.most_common(1000):
    ref,cnt  = item[0],item[1]
    artistID = dutils.getArtistID(ref)
    modVal   = str(int(artistID[-2:]))
    
    try:
        genre = artistMetaData[artistID]['Genre'].most_common(1)[0][0]
    except:
        continue
    #if genre not in ["Rock", "Pop", "Hip Hop"]:
    #    continue

    if toget.get(artistID) is None:
        toget[artistID] = {}
        
    
    coreAlbums = coreAlbumsRefs.get(artistID)
    #if len(coreAlbums) % 25 == 0:
    #print("{0: <50}{1: <30}{2: <10}{3}".format(ref,genre,artistID,len(coreAlbums)))
        
    for albumID,albumRef in coreAlbums.items():
        haveIt = albumDB.get(albumID)
        #print("\t{0: <15}{1}".format(albumID,haveIt))
        if haveIt is None:
            toget[artistID][albumID] = albumRef
            
    print("{0: <15}{1: <30}{2: <5}{3: <5}".format(artistID, genre, len(toget[artistID]), len(coreAlbums)))
        
    continue
    if False:
        print(artistID,'\t',ref)
        arts.rmIDFromDB(artistID)
        url = arts.getArtistURL(ref)
        savename = arts.getArtistSavename(artistID)
        arts.downloadArtistURL(url=url, savename=savename)
        sleep(2)
    #print(artistID,item,len(coreAlbums))

In [ ]:
len(toget)

In [ ]:
known = {}

In [ ]:
# Ignore 0-311 (start with Carl Perkins)
known = getFile("ignoreAlbum.p")
for ia,(artistID,albumRefs) in enumerate(toget.items()):
    print("\n|\n| {0} \t {1}\n|\n".format(ia,artistID))
    for i,(albumID,albumRef) in enumerate(albumRefs.items()):
        url = albs.getAlbumURL(albumRef)
            
        savename = albs.getAlbumSavename(artistID, albumID)
        if isFile(savename):
            continue

        if ia != 563:
            if known.get(url) is True:
                continue

        if False:
            known[url] = True
            continue
            
        print(artistID,'\t',ia,'\t',len(toget),'\t\t',i,'/',len(albumRefs),'\t',artistID)
        print("\tGetting: {0}".format(savename))
        retval = albs.downloadAlbumURLData(url=url, savename=savename, artistID=artistID, sleeptime=1.75)
        if retval is False:
            known[url] = True
            if len(known) % 5 == 0:
                print("\n====>    There are now {0} known downloaded albums    <====\n".format(len(known)))
                saveFile(idata=known, ifile="ignoreAlbum.p")

In [ ]:
len(known)

In [ ]:
saveFile(idata=known, ifile="ignoreAlbum.p")

In [ ]:
known = getFile("ignoreAlbum.p")

In [ ]:
for artistID,artistRefs in toget.items():
    print(artistID,len(artistRefs))

In [ ]:
tmp = disc.getAlbumIDToArtistIDData()

In [ ]:
db30 = disc.getAlbumsDBModValData(44)
from collections import Counter

In [ ]:
artistMetaData = {}
for artistID,artistData in db30.items():
    print(artistID)
    artistMetaData[artistID] = {"Genre": Counter(), "Artists": Counter(), "Style": Counter()}
    for albumID,albumData in artistData.items():
        genres = albumData.profile.genre
        if not isinstance(genres, list):
            genres = [genres]
        for genre in genres:
            artistMetaData[artistID]['Genre'][genre.name] += 1
        
        artists = albumData.artist.artists
        for artist in artists:
            artistMetaData[artistID]['Artists'][artist.name] += 1
        
        styles = albumData.profile.style
        if not isinstance(styles, list):
            styles = [styles]
        for style in styles:
            artistMetaData[artistID]['Style'][style.name] += 1

In [ ]:
artistMetaData

In [ ]:
albdata = albs.getData(ifile)

In [ ]:
for k,v in albdata.get().items():
    print(k,'\t',v.get())

In [ ]:
ArtistAlbums = disc.getArtistIDCoreAlbumIDs()
AlbumIDToName = disc.getAlbumIDToNameData()

In [ ]:
from string import ascii_uppercase, digits, punctuation
vals = "{0}{1}{2}".format(ascii_uppercase, digits, punctuation)
AsciiToName = {val: [x for x in NameToIDs.keys() if (x.startswith(val) or x.startswith(val.lower()))] for val in vals}

In [ ]:
ddb = discdb(disc)
ddb.loadArtistDBs(IDToName=IDToName, NameToIDs=NameToIDs, AsciiToName=AsciiToName)
ddb.loadAlbumDBs(ArtistAlbums=ArtistAlbums, AlbumIDToName=AlbumIDToName)
ddb.findArtist("Prince", num=1)
ddb.findArtistAlbums()

In [ ]:
%load_ext autoreload
%autoreload
disc   = discogs()
arts   = artists(disc)
albs   = albums(disc)
dutils = discogsUtils()
col    = collections(disc)

for modVal in list(range(99, 50, -1)):
    albs.downloadAlbumModValData(modVal, maxAlbums=1)

In [ ]:
IDToName    = disc.getArtistIDToNameData()
IDToRef     = disc.getArtistIDToRefData()
AlbIDToName = disc.getAlbumIDToNameData()
AlbIDToRef  = disc.getAlbumIDToRefData()

IDToCoreAlbID = disc.getArtistIDCoreAlbumIDs()
IDToAlbID     = disc.getArtistIDAlbumIDs()

In [ ]:
df = DataFrame.fromIDToCoreAlbID

In [ ]:
x   = {k: ":".join(v) for k,v in IDToCoreAlbID.items()}
pdf = DataFrame.from_dict(x, orient='index')

In [ ]:
pdf

In [ ]:
from itertools import islice
def takeN(n, iterable):
    if isinstance(iterable, dict):
        "Return first n items of the iterable as a list"
        #keys = takeN(n, iterable.keys())
        keys = list(islice(iterable, n))        
        return {k: iterable[k] for k in keys}
    elif isinstance(iterable, list):
        "Return first n items of the iterable as a list"
        return list(islice(iterable, n))
    else:
        print("No idea about {0}".format(type(iterable)))
        return None


#n_items = takeN(500000, ddb.NameToIDs.keys())
#n_items = ddb.NameToIDs.keys()

In [ ]:
from itertools import islice
def takeN(n, iterable):
    "Return first n items of the iterable as a list"
    return list(islice(iterable, n))

n_items = takeN(500000, ddb.NameToIDs.keys())
n_items = ddb.NameToIDs.keys()

In [ ]:
db38 = disc.getArtistsDBModValData(38)
db38['1032538']

In [ ]:
db38['1032538'].artist.get()

In [ ]:
artistIDtoName  = disc.getArtistIDToNameData()

In [ ]:
artistIDtoName['1032538']

In [ ]:
adc = arts.getData('/Volumes/Music/Discog/artists/38/1032538.p')
adc.artist.get()

In [ ]:
#col.createCollectionsData()
#col.downloadCollections()
#col.parseCollections()
#col.mergeCollections(debug=True, force=True)
#col.createCollectionDBs(debug=True)

In [ ]:
disc.moveAlbumFilesToNewModValue(newModValue=100, oldModValue=500)

# Artist Section

In [ ]:
db = disc.getArtistNameToIDsData()

In [ ]:
#arts = artists(disc)
#art.findKnownArtists(debug=True)
#art.findUnknownArtists(minVal=200, debug=True)
#art.downloadUnknownArtists(debug=True)
#arts.parseArtistFiles()
#art.searchDiscogForArtist("Melanie Sparks")
#arts.buildMetadata()

# Albums Metadata

In [ ]:
artistNameToID   = disc.getArtistNameToIDData()
artistNameToIDs  = disc.getArtistNameToIDsData()
artistIDToName   = disc.getArtistIDToNameData()        
artistRefToID    = disc.getArtistRefToIDData()
artistIDToRef    = disc.getArtistIDToRefData()
artistRefToName  = disc.getArtistRefToNameData()
artistNameToRef  = disc.getArtistNameToRefData()
artistNameToRefs = disc.getArtistNameToRefsData()

In [ ]:
albumRefToName  = disc.getAlbumRefToNameData()
albumNameToRef  = disc.getAlbumNameToRefData()

artistIDCoreAlbumIDs   = disc.getArtistIDCoreAlbumNames()
artistIDAlbumIDs       = disc.getArtistIDAlbumNames()



In [ ]:
from collections import Counter
core = ["Albums"]
albumCntr = Counter()


print("{0: <4}{1: <50}{2: <15}{3: <15}{4: <12}{5: <12}{6: <12}{7: <12}{8: <12}{9: <12}".format(
    "#","File","Core IDs-IDs","IDs-IDs","Name-IDs","Name-Refs","ID-Names","ID-Refs","Ref-IDs","Ref-Names"))


for i,dbfile in enumerate(dbfiles):
    db = getFile(dbfile)
    for artistID,artistData in db.items():
        artist      = artistData['Artist']
        artistName  = dutils.getArtistName(artist)
        artistRef   = artistData['URL']
        #print("{0: <20}{1: <50}{2}".format(artistID,artist,artistRef))
        
    print("{0: <4}{1: <50}{2: <15}{3: <15}{4: <12}{5: <12}{6: <12}{7: <12}{8: <12}{9: <12}".format(
        i,dbfile,
        len(artistIDCoreAlbumIDs),len(artistIDAlbumIDs),
        len(albumNameToID),len(albumNameToRef),len(albumIDToName),
        len(albumIDToRef),len(albumRefToID),len(albumRefToName)))



In [ ]:
ifile="/Volumes/Music/Discog/artists/0/9900.p"
fdata = getHTML(getFile(ifile, version=2))
an = artistName(fdata)
an.getArtist()
#name = artistName(fdata)
#name


In [ ]:
cnts   = [x.split()[0] for x in vals.split("\n")]
styles = [" ".join(x.split()[1:]) for x in vals.split("\n")]
d2000  = dict(zip(cnts, styles))
saveFile(idata=d2000, ifile="soundtracks.p")

# Albums

In [ ]:
albs = albums(disc)
albs.parseAlbumFiles()

In [ ]:
getFile('/Volumes/Music/Discog/albums-db/NAN-DB.p')

In [ ]:
import random
albs = albums(disc)
nErrs = False
while nErrs is False:
    nErrs = True
    albs.downloadAlbumDataData(int(random.uniform(0,disc.getMaxModVal()))) #, mediaTypes=None)        
    if False:
        nErrs = False
        print("Sleeping for 60 seconds...")
        sleep(60)
#art.findKnownArtists(debug=True)
#art.findUnknownArtists(minVal=200, debug=True)
#art.downloadUnknownArtists(debug=True)
#arts.parseArtistFiles()
#art.searchDiscogForArtist("$uicideboy$")
#arts.buildMetadata()

In [ ]:
%load_ext autoreload
%autoreload

from artist import artist
from artists import artists
art = artist()
arts = artists(disc)

toget=getFile(ifile="toget3.p")
done = {}
i = 0
errs = {}
n = len(toget)
for artistID,artistRef in toget.items():
    i += 1
    ifile = arts.getArtistSavename(artistID)
    print(i,'/',n,'\t',ifile)
    info  = art.getData(ifile)
    pages = info.pages
    
    if pages.redo is True:
        url      = arts.getArtistURL(artistRef, 1)
        savename = arts.getArtistSavename(artistID, 1)
        try:
            arts.downloadArtistURL(url=url, savename=savename, force=True, debug=True)            
        except:
            errs[artistID] = artistRef
            print("\tErr with {0}".format(artistRef))

In [ ]:
db = disc.getArtistsDBModValData(38)